In [2]:
import os
import time
import datetime
import numpy as np
from PIL import Image
from IPython.display import display, Markdown
import struct
import random
from subprocess import call
from tqdm.notebook import tqdm #progress bar
from numpy.linalg import inv
from skimage.measure import block_reduce
from skimage import io
from skimage.metrics import structural_similarity as ssim
# import cv2
import pandas as pd

In [3]:
def subfolder_size(input_folder):
    subfolders = [os.path.join(input_folder, d) for d in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, d))]

    total = 0
    
    for subfolder in subfolders:
        files = os.listdir(subfolder)
        for file in files:
            if os.path.basename(file).startswith('.'):
                    continue
            total += 1
    return total

In [4]:
def combine8x8(array):
    for i in range(32):
        for j in range(32):
            if j == 0:
                array4=array[i][j]
            else:
                array4=np.concatenate((array4,array[i][j]), axis=1)
        if i == 0:
            array5=array4
        else:
            array5=np.concatenate((array5,array4), axis=0)
    return array5

In [5]:
def rld(array,size):
    out=[]
    k=-1
    if size:
        val=(512*512)
    else:
        val=(256*256)
    vari=np.zeros((val), np.int16)
    for i in range(len(array)):
        k=k+1
        if k>=len(array) or len(out)>=val:
            break
        if array[k]==0:
            for j in range(array[k+1]):
                if len(out)>=val:
                    break
                out.append(array[k])
            k=k+1
        else:
            out.append(array[k])
    return out

In [6]:
def inverse_zigzag(inputarr):
    vmax=8
    hmax=8

    h = 0
    v = 0

    vmin = 0
    hmin = 0

    output = np.zeros((vmax, hmax))

    i = 0
    while ((v < vmax) and (h < hmax)): 
        if ((h + v) % 2) == 0:                 # going up
            if (v == vmin):
                output[v, h] = inputarr[i]        # if we got to the first line

                if (h == hmax):
                    v = v + 1
                else:
                    h = h + 1                        
                i = i + 1
            elif ((h == hmax -1 ) and (v < vmax)):   # if we got to the last column
                output[v, h] = inputarr[i] 
                v = v + 1
                i = i + 1
            elif ((v > vmin) and (h < hmax -1 )):    # all other cases
                output[v, h] = inputarr[i] 
                v = v - 1
                h = h + 1
                i = i + 1
        else:                                    # going down
            if ((v == vmax -1) and (h <= hmax -1)):       # if we got to the last line
                output[v, h] = inputarr[i] 
                h = h + 1
                i = i + 1
            elif (h == hmin):                  # if we got to the first column
                output[v, h] = inputarr[i] 
                if (v == vmax -1):
                    h = h + 1
                else:
                    v = v + 1
                i = i + 1
            elif((v < vmax -1) and (h > hmin)):     # all other cases
                output[v, h] = inputarr[i] 
                v = v + 1
                h = h - 1
                i = i + 1

        if ((v == vmax-1) and (h == hmax-1)):          # bottom right element
            output[v, h] = inputarr[i] 
            break
    return output

In [7]:
def ycbcr2rgb(im):
    xform = np.array([[1, 0, 1.402], [1, -0.34414, -.71414], [1, 1.772, 0]])
    rgb = im.astype(float)
    rgb[:,:,[1,2]] -= 128
    rgb[:,:,0] -= 16
    rgb = rgb.dot(xform.T)
    np.putmask(rgb, rgb > 255, 255)
    np.putmask(rgb, rgb < 0, 0)
    return np.uint8(rgb)

In [8]:
idct=inv(np.array([[0.3536,    0.3536,    0.3536,    0.3536,    0.3536,    0.3536,    0.3536,    0.3536],
    [0.4904,    0.4157,    0.2778,    0.0975,   -0.0975,   -0.2778,   -0.4157,   -0.4904],
    [0.4619,    0.1913,   -0.1913,   -0.4619,   -0.4619,   -0.1913,    0.1913,    0.4619],
    [0.4157,   -0.0975,   -0.4904,   -0.2778,    0.2778,    0.4904,    0.0975,   -0.4157],
    [0.3536,   -0.3536,   -0.3536,    0.3536,    0.3536,   -0.3536,   -0.3536,    0.3536],
    [0.2778,   -0.4904,    0.0975,    0.4157,   -0.4157,   -0.0975,    0.4904,   -0.2778],
    [0.1913,   -0.4619,    0.4619,   -0.1913,   -0.1913,    0.4619,   -0.4619,    0.1913],
    [0.0975,   -0.2778,    0.4157,   -0.4904,    0.4904,   -0.4157,    0.2778,   -0.0975]]))
dct=np.array([[0.3536,    0.3536,    0.3536,    0.3536,    0.3536,    0.3536,    0.3536,    0.3536],
    [0.4904,    0.4157,    0.2778,    0.0975,   -0.0975,   -0.2778,   -0.4157,   -0.4904],
    [0.4619,    0.1913,   -0.1913,   -0.4619,   -0.4619,   -0.1913,    0.1913,    0.4619],
    [0.4157,   -0.0975,   -0.4904,   -0.2778,    0.2778,    0.4904,    0.0975,   -0.4157],
    [0.3536,   -0.3536,   -0.3536,    0.3536,    0.3536,   -0.3536,   -0.3536,    0.3536],
    [0.2778,   -0.4904,    0.0975,    0.4157,   -0.4157,   -0.0975,    0.4904,   -0.2778],
    [0.1913,   -0.4619,    0.4619,   -0.1913,   -0.1913,    0.4619,   -0.4619,    0.1913],
    [0.0975,   -0.2778,    0.4157,   -0.4904,    0.4904,   -0.4157,    0.2778,   -0.0975]])

In [9]:
q=np.array([[16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12,14,19,26,58,60,55],
        [14, 13, 16,24,40,57,69,56],
        [14, 17, 22,29,51,87,80,62],
        [18, 22, 37,56,68,109,103,77],
        [24, 35, 55,64,81,104,113,92],
        [49, 64, 78,87,103,121,120,101],
        [72, 92, 95,98,112,100,103,99]])

q2=np.array([# 50% chrominance
        [17, 18, 24, 47, 99, 99, 99, 99],
        [18, 21, 26, 66, 99, 99, 99,99],
        [24, 26, 56, 99, 99, 99, 99,99],
        [47, 66, 99, 99, 99, 99, 99,99],
        [99, 99, 99, 99, 99, 99, 99,99],
        [99, 99, 99, 99, 99, 99, 99,99],
        [99, 99, 99, 99, 99, 99, 99,99],
        [99, 99, 99, 99, 99, 99, 99,99]])

qlum_75 = np.array([
 [ 8,     6,     5,     8,    12,    20,    26,    31],
 [ 6,     6,     7,    10,    13,    29,    30,    28],
 [ 7,     7,     8,    12,    20,    29,    35,    28],
 [ 7,     9,    11,    15,    26,    44,    40,    31],
 [ 9,    11,    19,    28,    34,    55,    52,    39],
 [12,    18,    28,    32,    41,    52,    57,    46],
 [25,    32,    39,    44,    52,    61,    60,    51],
 [36,    46,    48,    49,    56,    50,    52,    50]])


qchr_75 = np.array([
[9,     9,     12,    24,    50,    50,    50,    50],
[9,     11,    13,    33,    50,    50,    50,    50],
[12,    13,    28,    50,    50,    50,    50,    50],
[24,    33,    50,    50,    50,    50,    50,    50],
[50,    50,    50,    50,    50,    50,    50,    50],
[50,    50,    50,    50,    50,    50,    50,    50],
[50,    50,    50,    50,    50,    50,    50,    50],
[50,    50,    50,    50,    50,    50,    50,    50]])

qlum_50 = np.array([
[0x10, 0x0B, 0x0C, 0x0E, 0x0C, 0x0A, 0x10, 0x0E], 
[0x0D, 0x0E, 0x12, 0x11, 0x10, 0x13, 0x18, 0x28],
[0x1A, 0x18, 0x16, 0x16, 0x18, 0x31, 0x23, 0x25], 
[0x1D, 0x28, 0x3A, 0x33, 0x3D, 0x3C, 0x39, 0x33],
[0x38, 0x37, 0x40, 0x48, 0x5C, 0x4E, 0x40, 0x44], 
[0x57, 0x45, 0x37, 0x38, 0x50, 0x6D, 0x51, 0x57],
[0x5F, 0x62, 0x67, 0x68, 0x67, 0x3E, 0x4D, 0x71], 
[0x79, 0x70, 0x64, 0x78, 0x5C, 0x65, 0x67, 0x63]])

qchr_50 = np.array([
[0x11, 0x12, 0x12, 0x18, 0x15, 0x18, 0x2F, 0x1A], 
[0x1A, 0x2F, 0x63, 0x42, 0x38, 0x42, 0x63, 0x63],
[0x63, 0x63, 0x63, 0x63, 0x63, 0x63, 0x63, 0x63], 
[0x63, 0x63, 0x63, 0x63, 0x63, 0x63, 0x63, 0x63],
[0x63, 0x63, 0x63, 0x63, 0x63, 0x63, 0x63, 0x63], 
[0x63, 0x63, 0x63, 0x63, 0x63, 0x63, 0x63, 0x63],
[0x63, 0x63, 0x63, 0x63, 0x63, 0x63, 0x63, 0x63], 
[0x63, 0x63, 0x63, 0x63, 0x63, 0x63, 0x63, 0x63]])

qlum_25 = np.array([
  [32,    22,    20,    32,    48,    80,    102,   122],
  [24,    24,    28,    38,    52,    116,   120,   110],
  [28,    26,    32,    48,    80,    114,   138,   112],
  [28,    34,    44,    58,    102,   174,   160,   124],
  [36,    44,    74,    112,   136,   218,   206,   154],
  [48,    70,    110,   128,   162,   208,   226,   184],
  [98,    128,   156,   174,   206,   242,   240,   202],
  [144,   184,   190,   196,   224,   200,   206,   198]])


qchr_25 = np.array([
    [34,    36,    48,    94,   198,   198,   198,   198],
    [36,    42,    52,   132,   198,   198,   198,   198],
    [48,    52,   112,   198,   198,   198,   198,   198],
    [94,   132,   198,   198,   198,   198,   198,   198],
   [198,   198,   198,   198,   198,   198,   198,   198],
   [198,   198,   198,   198,   198,   198,   198,   198],
   [198,   198,   198,   198,   198,   198,   198,   198],
   [198,   198,   198,   198,   198,   198,   198,   198]])

qchr_20 = np.array([
    [43,    45,    60,   118,   248,   248,   248,   248],
    [45,    53,    65,   165,   248,   248,   248,   248],
    [60,    65,   140,   248,   248,   248,   248,   248],
   [118,   165,   248,   248,   248,   248,   248,   248],
   [248,   248,   248,   248,   248,   248,   248,   248],
   [248,   248,   248,   248,   248,   248,   248,   248],
   [248,   248,   248,   248,   248,   248,   248,   248],
   [248,   248,   248,   248,   248,   248,   248,   248]])

qlum_20 = np.array([
    [40,    28,    25,    40,    60,   100,   128,   153],
    [30,    30,    35,    48,    65,   145,   150,   138],
    [35,    33,    40,    60,   100,   143,   173,   140],
    [35,    43,    55,    73,   128,   218,   200,   155],
    [45,    55,    93,   140,   170,   273,   258,   193],
    [60,    88,   138,   160,   203,   260,   283,   230],
   [123,   160,   195,   218,   258,   303,   300,   253],
   [180,   230,   238,   245,   280,   250,   258,   248]])
qchr_0 = np.array([
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255]])

qlum_0 = np.array([
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255],
   [255,   255,   255,   255,   255,   255,   255,   255]])

In [10]:
def chroma_read(inputarr):
    temp=np.zeros((256,256), np.int16)
    for k in range(32):
        for l in range(32):
            for i in range(8):
                for j in range(8):
                    temp[8*k+i][8*l+j]=inputarr[8*k+i][8*l+j]*q2[i][j]
    temp1=np.zeros((256,256), np.int16)
    for k in range(32):
        for l in range(32):
            for i in range(8):
                for j in range(8):
                    for p in range(8):
                        temp1[8*k+i][8*l+j]+=(temp[8*k+i][8*l+p]*idct[j][p])
    temp2=np.zeros((256,256), np.int16)
    for k in range(32):
        for l in range(32):
            for i in range(8):
                for j in range(8):
                    for p in range(8):
                        temp2[8*k+i][8*l+j]+=(idct[i][p]*temp1[8*k+p][8*l+j])
    return temp, temp1, temp2

In [12]:
# while 1:
input_folder2 = "camera"
input_folder = "/Volumes/xilinx/jupyter_notebooks/Labs/Lab24/camera"
output_folder = "images"
# luminance_quality=np.array([qlum_75, qlum_50, qlum_25, qlum_0])
# chroma_quality=np.array([qchr_75, qchr_50, qchr_25, qchr_0])
subfolders = [os.path.join(input_folder, d) for d in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, d))]
outsubfolders = [os.path.join(f'{output_folder}/{input_folder2}', d) for d in os.listdir(f'{output_folder}/{input_folder2}') if os.path.isdir(os.path.join(f'{output_folder}/{input_folder2}', d))]
# List all files in the input folder
# subsubfolders2 = [os.path.join(os.path.join(subfolder,"imagenet_subset"), d) for d in os.listdir(os.path.join(subfolders[0],"imagenet_subset")) if os.path.isdir(os.path.join(os.path.join(subfolder,"imagenet_subset"), d))]
with tqdm(
    total=(4*subfolder_size(os.path.join(subfolders[0],"imagenet_subset"))), #-(4*subfolder_size(os.path.join(outsubfolders[0],"imagenet_subset")))
    # smoothing=0,
    unit=" images",
    bar_format="{l_bar}{bar}\t{n_fmt}/{total_fmt}\t[ time left: {remaining}, rate: {rate_fmt}{postfix}]"
) as pbar:
    for subfolder in subfolders:
        #Check for hidden files and folders
        if os.path.basename(subfolder).startswith('.'):
            continue

        # Loop through each file in the subfolder

        subsubfolders2 = [os.path.join(os.path.join(subfolder,"imagenet_subset"), d) for d in os.listdir(os.path.join(subfolder,"imagenet_subset")) if os.path.isdir(os.path.join(os.path.join(subfolder,"imagenet_subset"), d))]
        for subfolder2 in subsubfolders2:
        #Check for hidden files and folders
            if os.path.basename(subfolder2).startswith('.'):
                continue
            files = os.listdir(subfolder2)
            for file in files:
                if os.path.basename(file).startswith('.'):
                    continue
                file_name, ext = os.path.splitext(file)
                if ext.lower() not in ['.npy']:
                    continue
                output_path = f'{output_folder}/{subfolder2}'
                if not os.path.exists(output_path): #Create the category folder
                    os.makedirs(output_path)
                

                with open(f'{subfolder2}/{file}', 'rb') as f:
                    Y_encoded = np.load(f)
                    Cb_encoded = np.load(f)
                    Cr_encoded = np.load(f)
                    original_image_size = np.load(f)
                    qual = np.load(f)[0]

                # if os.path.exists(f'{output_path}/{file_name}.JPEG'):
                #     pbar.update(1)
                #     continue
                # print(f'{output_path}/{file_name}.JPEG')
                if not os.path.exists(f'{output_folder}/{input_folder2}/{qual}/imagenet_subset/{os.path.basename(subfolder2)}'): #Create the category folder
                    os.makedirs(f'{output_folder}/{input_folder2}/{qual}/imagenet_subset/{os.path.basename(subfolder2)}')



                # if os.path.exists(f'{output_folder}/{input_folder2}/{qual}/imagenet_subset/{os.path.basename(subfolder2)}/{file_name}.JPEG'):
                #     pbar.update(1)
                #     continue


                Y_decoded=np.array(rld(Y_encoded,True))
                Cb_decoded=np.array(rld(Cb_encoded,False))
                Cr_decoded=np.array(rld(Cr_encoded,False))
                if qual==0:
                    q=qlum_0
                    q2=qchr_0
                elif qual==25:
                    q=qlum_25
                    q2=qchr_25
                elif qual==50:
                    q=qlum_50
                    q2=qchr_50
                elif qual==75:
                    q=qlum_75
                    q2=qchr_75
                result1=np.zeros((32,32,16,16), np.int16)
                if np.max(Cb_decoded)>0 and np.max(Cr_decoded)>0:
                    result2=np.zeros((32,32,8,8), np.int16)
                    result3=np.zeros((32,32,8,8), np.int16)

                r1=0
                r2=0
                for i in range(32):
                    for j in range(32):
                        r1=np.concatenate((inverse_zigzag(Y_decoded[256*(i*32+j):256*(i*32+j)+64]),inverse_zigzag(Y_decoded[256*(i*32+j)+64:256*(i*32+j)+128])), axis=1)
                        r2=np.concatenate((inverse_zigzag(Y_decoded[256*(i*32+j)+128:256*(i*32+j)+192]),inverse_zigzag(Y_decoded[256*(i*32+j)+192:256*(i*32+j)+256])), axis=1)
                        result1[i][j]=np.concatenate((r1,r2), axis=0)
                        if np.max(Cb_decoded)>0 and np.max(Cr_decoded)>0:
                            result2[i][j]=inverse_zigzag(Cb_decoded[64*(i*32+j):64*(i*32+j)+64])
                            result3[i][j]=inverse_zigzag(Cr_decoded[64*(i*32+j):64*(i*32+j)+64])
                Y_result=combine8x8(result1)
                if np.max(Cb_decoded)>0 and np.max(Cr_decoded)>0:
                    Cb_result=combine8x8(result2)
                    Cr_result=combine8x8(result3)
                    Cbchroma=chroma_read(Cb_result)
                    Crchroma=chroma_read(Cr_result)

                new_Y_output=np.zeros((512,512), np.int16)
                for k in range(64):
                    for l in range(64):
                        for i in range(8):
                            for j in range(8):
                                new_Y_output[8*k+i][8*l+j]=Y_result[8*k+i][8*l+j]*q[i][j]
                newer_Y_output=np.zeros((512,512), np.int16)
                for k in range(64):
                    for l in range(64):
                        for i in range(8):
                            for j in range(8):
                                for p in range(8):
                                    newer_Y_output[8*k+i][8*l+j]+=(new_Y_output[8*k+i][8*l+p]*idct[j][p])
                newest_Y_output=np.zeros((512,512), np.int16)
                for k in range(64):
                    for l in range(64):
                        for i in range(8):
                            for j in range(8):
                                for p in range(8):
                                    newest_Y_output[8*k+i][8*l+j]+=(idct[i][p]*newer_Y_output[8*k+p][8*l+j])
                finalY=np.zeros((512,512), np.uint8)
                finalCb=np.zeros((256,256), np.uint8)
                finalCr=np.zeros((256,256), np.uint8)
                finalY=(newest_Y_output+128).astype(np.uint8)
                if np.max(Cb_decoded)>0 and np.max(Cr_decoded)>0:
                    finalCb=(Cbchroma[2]+128).astype(np.uint8)
                    finalCr=(Crchroma[2]+128).astype(np.uint8)
                    n = 2
                    upscaleCb=np.kron(finalCb, np.ones((n,n)))
                    upscaleCr=np.kron(finalCr, np.ones((n,n)))

                if np.max(Cb_decoded)>0 and np.max(Cr_decoded)>0:
                    upscale_newer_array=np.zeros((512,512,3))
                    for i in range(512):
                        for j in range(512):
                            upscale_newer_array[i][j]=[finalY[i][j], upscaleCr[i][j], upscaleCb[i][j]]
                    compressed_image = Image.fromarray(ycbcr2rgb(upscale_newer_array).astype(np.uint8))
                    original_compressed_image = compressed_image.resize((original_image_size[0], original_image_size[1]), Image.BILINEAR)
                else:
                    upscale_newer_array=np.zeros((512,512))
                    for i in range(512):
                        for j in range(512):
                            upscale_newer_array[i][j]=finalY[i][j] 
                    compressed_image = Image.fromarray(upscale_newer_array.astype(np.uint8), "L")
                    original_compressed_image = compressed_image.resize((original_image_size[0], original_image_size[1]), Image.BILINEAR)
                # display(compressed_image)
                # display(original_compressed_image)
                # print("OUT", f'{output_folder}/{os.path.join(subfolder,subfolder2)}{file_name}.JPEG') 
                original_compressed_image.save(f'{output_folder}/{input_folder2}/{qual}/imagenet_subset/{os.path.basename(subfolder2)}/{file_name}.JPEG')
                pbar.update(1)
                # return original_compressed_image

  0%|          	0/16	[ time left: ?, rate: ? images/s]

In [ ]:
path_to_watch = "/Volumes/xilinx/jupyter_notebooks/Labs/Lab24/arrays"
print('Your folder path is "',path_to_watch,'"')
before = dict ([(f, None) for f in os.listdir (path_to_watch)])
while 1:
        after = dict ([(f, None) for f in os.listdir (path_to_watch)])
        added = [f for f in after if not f in before]
        if added:
                print("Added: ", ", ".join (added))
                break
        else:
             before = after

Your folder path is " /Volumes/xilinx/jupyter_notebooks/Labs/Lab24/arrays "


KeyboardInterrupt: 

In [ ]:
os.listdir ("/Volumes/xilinx/jupyter_notebooks/Labs/Lab24/arrays")

['25', '50', '0', '75']